# Medición de longitud 
sin considerar distorsión

In [ ]:
import seaborn as sb
import numpy as np
import cv2
def graficar2(listImgs,nrow,ncol,listTitles):
    nImg=len(listImgs)
    nTitles=len(listTitles)
    if nTitles!=0 and nTitles!=nImg:
        print("numero de titulos incorrecto")
    fig, ax = plt.subplots(nrow,ncol,figsize=(20,7))
    
    for i,img in enumerate(listImgs):
        heat_map = sb.heatmap(img,robust=True,ax=ax[i],yticklabels=False,xticklabels=False)

        if nTitles>0 : 
            ax[i].set_title(listTitles[i],fontsize=20)

    plt.show()

In [ ]:
import numpy as np
import cv2
import sys# las sisguientes dos lineas ayudan a ubicar al archivo calibracion.py
sys.path.append("/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/")
from calibracion import graficar
from medicion import *
class longitud0(imagen,puntos):
    # path_configuracion: dirección absoluta del lugar donde se encuentra el archivo de configuración
    #nombre_configuración: nombre del archivo configuración sin la extensión .ini
    #d:diametro de circulos en milimetros
    # nf: numero de filas del patron
    #nc: numero de circulos por fila del patron
    def __init__(self,path_configuracion,nombre_configuracion,d,nf,nc):
        imagen.__init__(self,path_configuracion,nombre_configuracion,pendientes2=False)
        puntos.__init__(self,nf,nc)
        
        # variables de isntancia
        self.distancia_patron=self.formar_distancia(d)# distancia original entre puntos contiguos del patron. tiene en cuenta que los puntos estan agrupados por fila, los puntos de cada fila estan ordenados de izquierda a derecha, y las filas de arriba hacia abajo.
        self.error_promedio=0
    # abre el par de imagenes rgb y profundidad que esten captando la misma escena
    #los nombre desben estar en el formato indicador_nombre. indicardor="rgb" para la imagen del sensor rgb
    #indicador="depth" para la imagen del sensor de profundidad. 
    #nobre debera ser el mismo para el par de imagenes que representen la misma escena.
    #pasa la imagen rgb al dominio de profundidad
    #
    def obtener_uvz(self,path, indicador1,indicador2,nombre):
        #abrir imagenes
        ret,img_rgb,img_depth=self.abrir_par_img(path,indicador1,indicador2,nombre)
        img_depth=self.recuperar(img_depth)
        #quitar distorision z
#         img_depth=self.distorsion_z(img_depth,filtrar=True)
        #quitar distorsion
        img_rgb,img_depth=self.distorsion(img_rgb,img_depth)
        
        # cambiar de dominio
        img2_depth=self.rgb2_depth(img_rgb)
        # hallar puntos medios entre los circulos de una misma fila
        ret,_,puntos_uv=self.extraer(img2_depth,0)# es una imagen en el dominio de profundidad pero su caracteristicas siguen siendo de color. no necesita mejora
        # informar si hubo detección de puntos
        print("hubo de detección de puntos?: {}".format(ret))
        #obtener cordenada z # numpy indexa fila x columna, opencv indexa ancho por lato. por lo tanto hay que invertir los indices.
        if ret:
            z=[]
            for punto in puntos_uv:
                fila=int(np.around(punto[0][1]))
                col=int(np.around(punto[0][0]))
                valor=img_depth[fila,col]

                z.append(valor)
            z=np.array([z])
        #dibujar puntos medios sobre la imagen de profundidad
        img_puntos=self.dibujar(img_depth,puntos_uv,1)
        return ret,puntos_uv,z,img_puntos
    # la función resive puntos uv y los trasforma a puntos X,Y,Z
    # reproyecta los puntos uv encontrados en la imagen y los lleva al espacio.las cordenadas son cartesians y su origen  es el foco de la camara
    def obtener_xyz(self,puntos_uv,z):
        
        # los puntos estan almacenados en vectores fila [u,v]. para la multiplicacion matricial se los lleva a vectores tipo columna [u;v;1]
        puntos_uv=puntos_uv.reshape(-1,2) # desepaquetar. de shape(n,1,2) hacia shape(n,2)
        puntos_u=puntos_uv[:,0]# extraer todas las componentes u de los puntos
        puntos_v=puntos_uv[:,1]# #extraer todas las componentes v
        unos=np.ones(puntos_uv.shape[0])
        puntos_uv1=np.array((puntos_u,puntos_v,unos))# formar vectores columna (u;v;1)
        puntos_xy1_n=np.dot(self.depth_inversa,puntos_uv1)# calcular cordenadas normalizadas
        puntos_xyz=z*puntos_xy1_n# calular cordenadas espaciales de la forma [X;Y;Z]. son vectores columna
        #devolver puntos a forma vector fila [(x,y,z)].separando los putos
        puntos_xyz=np.concatenate((puntos_xyz[0].reshape(-1,1),puntos_xyz[1].reshape(-1,1),puntos_xyz[2].reshape(-1,1)),axis=1)
        return (puntos_xyz) #entregar array de la forma shape(n,3)
    #encuentra la distancia euclidiana entre los puntos contiguos XYZ y los compara  con la distancia de construcción de los puntos del patron
    def calcular_distancia(self,puntos_xyz):
        distancias=[]
        error_distancias=[]
        for i in range(puntos_xyz.shape[0]-1):
            vector=puntos_xyz[i+1]-puntos_xyz[i]
            distancia=np.sqrt(np.sum(np.power(vector,2)))
            distancias.append(distancia)
        distancias=np.array(distancias).reshape(-1,1)/10
        error_distancias=np.absolute(self.distancia_patron-distancias)
        error_distancias=100*error_distancias/self.distancia_patron
        self.error_promedio=np.sum(error_distancias)/error_distancias.size
        return distancias,error_distancias,self.error_promedio
    #la distancia se calcula entre dos puntos contiguos
    #d=diametro de los circulos
    #nf=numero de filas del patron
    #nc=numero de circulos por fila
    #los puntos estan agrupados por filas, los puntos de cada fila se encuentran ordenados de izquierda a derecha y las filas de arriba hacia abajo
    def formar_distancia(self,d):
        nc=self.nc-1# al ser puntos medio entre circulos se pierde un punto por cada fila
        distancias_patron=np.ones((self.nf*nc-1,1))#al ser distancia sobre puntos contiguos la informacion disminuye en un dato
        #formar distancias teniendo en cuenta la estructura del patron
        for i in range(distancias_patron.size):
            if (i+1)%nc:# puntos contiguos de una misma fila
                distancias_patron[i]=2*d
            else:#puntos contiguos, punto extremo derecho fila superior, punto extremo izquierdo fila inferior
                if (i+1)%(2*nc):# por se patron asimetrico,las distancias entre extremos cambian de una fila a otra
                    distancias_patron[i]=np.sqrt((2*(nc-1)*d-d)**2+d**2)#fila impar
                else:
                    distancias_patron[i]=np.sqrt((2*(nc-1)*d+d)**2+d**2)#fila par
        return distancias_patron


In [ ]:
#from medicion import *
import sys# las sisguientes dos lineas ayudan a ubicar al archivo calibracion.py
sys.path.append("/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/")
from calibracion import *
path_configuracion="/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuraciónPrueba/"
nombre_configuracion="d5_90_union_150"
path_img="/home/estufab4/Desktop/flujo de bagazo/codigo/img_medicion/longitud"
d=100
nf=8
nc=5
medir=longitud0(path_configuracion,nombre_configuracion,d,nf,nc)
ret,puntos_uv,z,img_puntos=medir.obtener_uvz(path_img,"gray","depth",9)

if ret:
        print("distancia media",z.mean())
        xyz=medir.obtener_xyz(puntos_uv,z)
        distancias0,error_distancias0,error_promedio0=medir.calcular_distancia(xyz)
        for i in range(distancias.size):
            print("puntos {} y {} : las siguientes son unidades en milimetros".format(i,i+1))
            print("distancia calculada:{}".format(distancias0[i]))
            print("distancia original:{}".format(medir.distancia_patron[i]))
            print("error en porcentaje :{}%".format(error_distancias0[i]))
        print("error promedio en porcentaje :{}%".format(error_promedio0))
        graficar([img_puntos],1,1,["imagen depth con puntos detectados"])



In [ ]:
from tabulate import tabulate
tabla=[]
for distancia,distancia_patron,error,i in zip(distancias0,medir.distancia_patron,error_distancias0,range(distancias0.size)):
    tabla.append(["{} y {}".format(i,i+1),distancia,distancia_patron,error])
print("error promedio",error_distancias0.mean())
print("Desviación error",error_distancias0.std())
print(tabulate(tabla,headers=["puntos","V medido[mm]","V Real[mm]","error%"]))
hist0,x0=np.histogram(error_distancias0,bins=20)
hist0=100*hist0/len(error_distancias0)
plt.hist(x0[:-1],x0, weights=hist0)
plt.title("Media: {:.2f}, STD= {:.2f}".format(error_distancias0.mean(),error_distancias0.std()))
plt.grid()

# medición de longitud
1. se tiene en cuenta la distorsión en z, teniendo en cuenta una curva mx+b

In [ ]:
import numpy as np
import cv2
import sys# las sisguientes dos lineas ayudan a ubicar al archivo calibracion.py
sys.path.append("/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/")
from calibracion import graficar
from medicion import *
from ttictoc import tic,toc
class longitud(imagen,puntos):
    # path_configuracion: dirección absoluta del lugar donde se encuentra el archivo de configuración
    #nombre_configuración: nombre del archivo configuración sin la extensión .ini
    #d:diametro de circulos en milimetros
    # nf: numero de filas del patron
    #nc: numero de circulos por fila del patron
    def __init__(self,path_configuracion,nombre_configuracion,d,nf,nc):
        imagen.__init__(self,path_configuracion,nombre_configuracion,pendientes2=False)
        puntos.__init__(self,nf,nc)
        
        # variables de isntancia
        self.distancia_patron=self.formar_distancia(d)# distancia original entre puntos contiguos del patron. tiene en cuenta que los puntos estan agrupados por fila, los puntos de cada fila estan ordenados de izquierda a derecha, y las filas de arriba hacia abajo.
        self.error_promedio=0
    # abre el par de imagenes rgb y profundidad que esten captando la misma escena
    #los nombre desben estar en el formato indicador_nombre. indicardor="rgb" para la imagen del sensor rgb
    #indicador="depth" para la imagen del sensor de profundidad. 
    #nobre debera ser el mismo para el par de imagenes que representen la misma escena.
    #pasa la imagen rgb al dominio de profundidad
    #
    def obtener_uvz(self,path, indicador1,indicador2,nombre):
        #abrir imagenes
        ret,img_rgb,img_depth=self.abrir_par_img(path,indicador1,indicador2,nombre)
        tic()
        img_depth=self.recuperar(img_depth)
        print(toc(),"recuperación")
        #quitar distorision z
        img_depth=self.distorsion_z(img_depth,filtrar=False)
        #quitar distorsion
        img_rgb,img_depth=self.distorsion(img_rgb,img_depth)
        
        # cambiar de dominio
        img2_depth=self.rgb2_depth(img_rgb)
        # hallar puntos medios entre los circulos de una misma fila
        ret,_,puntos_uv=self.extraer(img2_depth,0)# es una imagen en el dominio de profundidad pero su caracteristicas siguen siendo de color. no necesita mejora
        # informar si hubo detección de puntos
        print("hubo de detección de puntos?: {}".format(ret))
        #obtener cordenada z # numpy indexa fila x columna, opencv indexa ancho por lato. por lo tanto hay que invertir los indices.
        if ret:
            z=[]
            for punto in puntos_uv:
                fila=int(np.around(punto[0][1]))
                col=int(np.around(punto[0][0]))
                valor=img_depth[fila,col]

                z.append(valor)
            z=np.array([z])
        #dibujar puntos medios sobre la imagen de profundidad
        img_puntos=self.dibujar(img_depth,puntos_uv,1)
        return ret,puntos_uv,z,img_puntos
    # la función resive puntos uv y los trasforma a puntos X,Y,Z
    # reproyecta los puntos uv encontrados en la imagen y los lleva al espacio.las cordenadas son cartesians y su origen  es el foco de la camara
    def obtener_xyz(self,puntos_uv,z):
        
        # los puntos estan almacenados en vectores fila [u,v]. para la multiplicacion matricial se los lleva a vectores tipo columna [u;v;1]
        puntos_uv=puntos_uv.reshape(-1,2) # desepaquetar. de shape(n,1,2) hacia shape(n,2)
        puntos_u=puntos_uv[:,0]# extraer todas las componentes u de los puntos
        puntos_v=puntos_uv[:,1]# #extraer todas las componentes v
        unos=np.ones(puntos_uv.shape[0])
        puntos_uv1=np.array((puntos_u,puntos_v,unos))# formar vectores columna (u;v;1)
        puntos_xy1_n=np.dot(self.depth_inversa,puntos_uv1)# calcular cordenadas normalizadas
        puntos_xyz=z*puntos_xy1_n# calular cordenadas espaciales de la forma [X;Y;Z]. son vectores columna
        #devolver puntos a forma vector fila [(x,y,z)].separando los putos
        puntos_xyz=np.concatenate((puntos_xyz[0].reshape(-1,1),puntos_xyz[1].reshape(-1,1),puntos_xyz[2].reshape(-1,1)),axis=1)
        return (puntos_xyz) #entregar array de la forma shape(n,3)
    #encuentra la distancia euclidiana entre los puntos contiguos XYZ y los compara  con la distancia de construcción de los puntos del patron
    def calcular_distancia(self,puntos_xyz):
        distancias=[]
        error_distancias=[]
        for i in range(puntos_xyz.shape[0]-1):
            vector=puntos_xyz[i+1]-puntos_xyz[i]
            distancia=np.sqrt(np.sum(np.power(vector,2)))
            distancias.append(distancia)
        distancias=np.array(distancias).reshape(-1,1)/10
        error_distancias=np.absolute(self.distancia_patron-distancias)
        error_distancias=100*error_distancias/self.distancia_patron
        self.error_promedio=np.sum(error_distancias)/error_distancias.size
        return distancias,error_distancias,self.error_promedio
    #la distancia se calcula entre dos puntos contiguos
    #d=diametro de los circulos
    #nf=numero de filas del patron
    #nc=numero de circulos por fila
    #los puntos estan agrupados por filas, los puntos de cada fila se encuentran ordenados de izquierda a derecha y las filas de arriba hacia abajo
    def formar_distancia(self,d):
        nc=self.nc-1# al ser puntos medio entre circulos se pierde un punto por cada fila
        distancias_patron=np.ones((self.nf*nc-1,1))#al ser distancia sobre puntos contiguos la informacion disminuye en un dato
        #formar distancias teniendo en cuenta la estructura del patron
        for i in range(distancias_patron.size):
            if (i+1)%nc:# puntos contiguos de una misma fila
                distancias_patron[i]=2*d
            else:#puntos contiguos, punto extremo derecho fila superior, punto extremo izquierdo fila inferior
                if (i+1)%(2*nc):# por se patron asimetrico,las distancias entre extremos cambian de una fila a otra
                    distancias_patron[i]=np.sqrt((2*(nc-1)*d-d)**2+d**2)#fila impar
                else:
                    distancias_patron[i]=np.sqrt((2*(nc-1)*d+d)**2+d**2)#fila par
        return distancias_patron


In [ ]:
#from medicion import *
import sys# las sisguientes dos lineas ayudan a ubicar al archivo calibracion.py
sys.path.append("/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/")
from calibracion import *
path_configuracion="/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuraciónPrueba/"
nombre_configuracion="d5_90_union_150"
path_img="/home/estufab4/Desktop/flujo de bagazo/codigo/img_medicion/longitud"
d=100
nf=8
nc=5
medir=longitud(path_configuracion,nombre_configuracion,d,nf,nc)
ret,puntos_uv,z,img_puntos=medir.obtener_uvz(path_img,"gray","depth",9)

if ret:
        print("distancia media",z.mean())
        xyz=medir.obtener_xyz(puntos_uv,z)
        distancias,error_distancias,error_promedio=medir.calcular_distancia(xyz)
        for i in range(distancias.size):
            print("puntos {} y {} : las siguientes son unidades en milimetros".format(i,i+1))
            print("distancia calculada:{}".format(distancias[i]))
            print("distancia original:{}".format(medir.distancia_patron[i]))
            print("error en porcentaje :{}%".format(error_distancias[i]))
        print("error promedio en porcentaje :{}%".format(error_promedio))
        graficar([img_puntos],1,1,["imagen depth con puntos detectados"])




In [ ]:
from tabulate import tabulate
tabla=[]
for distancia,distancia_patron,error,i in zip(distancias,medir.distancia_patron,error_distancias,range(distancias.size)):
    tabla.append(["{} y {}".format(i,i+1),distancia,distancia_patron,error])
print("error promedio",error_distancias.mean())
print("Desviación error",error_distancias.std())
print(tabulate(tabla,headers=["puntos","V medido[mm]","V Real[mm]","error%"]))
hist,x=np.histogram(error_distancias,bins=20)
hist=100*hist/len(error_distancias)
plt.hist(x[:-1],x, weights=hist)
plt.title("Media: {:.2f}, STD= {:.2f}".format(error_distancias.mean(),error_distancias.std()))
plt.grid()

# medición de longitud
1. se tiene en cuenta la distorsión en z, teniendo en cuenta una curva m2x²+mx+b


In [ ]:
import time
tic()
time.sleep(3)
print(toc())

In [ ]:
import numpy as np
import cv2
import sys# las sisguientes dos lineas ayudan a ubicar al archivo calibracion.py
sys.path.append("/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/")
from calibracion import graficar
from medicion import *
from ttictoc import tic,toc
class longitud2(imagen,puntos):
    # path_configuracion: dirección absoluta del lugar donde se encuentra el archivo de configuración
    #nombre_configuración: nombre del archivo configuración sin la extensión .ini
    #d:diametro de circulos en milimetros
    # nf: numero de filas del patron
    #nc: numero de circulos por fila del patron
    def __init__(self,path_configuracion,nombre_configuracion,d,nf,nc):
        imagen.__init__(self,path_configuracion,nombre_configuracion,pendientes2=True)
        puntos.__init__(self,nf,nc)
        
        # variables de isntancia
        self.distancia_patron=self.formar_distancia(d)# distancia original entre puntos contiguos del patron. tiene en cuenta que los puntos estan agrupados por fila, los puntos de cada fila estan ordenados de izquierda a derecha, y las filas de arriba hacia abajo.
        self.error_promedio=0
    # abre el par de imagenes rgb y profundidad que esten captando la misma escena
    #los nombre desben estar en el formato indicador_nombre. indicardor="rgb" para la imagen del sensor rgb
    #indicador="depth" para la imagen del sensor de profundidad. 
    #nobre debera ser el mismo para el par de imagenes que representen la misma escena.
    #pasa la imagen rgb al dominio de profundidad
    #
    def obtener_uvz(self,path, indicador1,indicador2,nombre):
        #abrir imagenes
        ret,img_rgb,img_depth=self.abrir_par_img(path,indicador1,indicador2,nombre)
        tic()
        img_depth=self.recuperar(img_depth)
        print(toc(),"recuperación")
        #quitar distorision z
        tic()
        img_depth=self.distorsion_z2(img_depth,filtrar=False)
        print(toc(),"distorsión z")
        #quitar distorsion UV
        tic()
        img_rgb,img_depth=self.distorsion(img_rgb,img_depth)
        print(toc(),"distorsión uv")
        # cambiar de dominio
        tic()
        img2_depth=self.rgb2_depth(img_rgb)
        print(toc(),"fusión")
        # hallar puntos medios entre los circulos de una misma fila
        ret,_,puntos_uv=self.extraer(img2_depth,0)# es una imagen en el dominio de profundidad pero su caracteristicas siguen siendo de color. no necesita mejora
        # informar si hubo detección de puntos
        print("hubo de detección de puntos?: {}".format(ret))
        #obtener cordenada z # numpy indexa fila x columna, opencv indexa ancho por lato. por lo tanto hay que invertir los indices.
        if ret:
            z=[]
            for punto in puntos_uv:
                fila=int(np.around(punto[0][1]))
                col=int(np.around(punto[0][0]))
                valor=img_depth[fila,col]

                z.append(valor)
            z=np.array([z])
        #dibujar puntos medios sobre la imagen de profundidad
        img_puntos=self.dibujar(img_depth,puntos_uv,1)
        return ret,puntos_uv,z,img_puntos
    # la función resive puntos uv y los trasforma a puntos X,Y,Z
    # reproyecta los puntos uv encontrados en la imagen y los lleva al espacio.las cordenadas son cartesians y su origen  es el foco de la camara
    def obtener_xyz(self,puntos_uv,z):
        
        # los puntos estan almacenados en vectores fila [u,v]. para la multiplicacion matricial se los lleva a vectores tipo columna [u;v;1]
        puntos_uv=puntos_uv.reshape(-1,2) # desepaquetar. de shape(n,1,2) hacia shape(n,2)
        puntos_u=puntos_uv[:,0]# extraer todas las componentes u de los puntos
        puntos_v=puntos_uv[:,1]# #extraer todas las componentes v
        unos=np.ones(puntos_uv.shape[0])
        puntos_uv1=np.array((puntos_u,puntos_v,unos))# formar vectores columna (u;v;1)
        puntos_xy1_n=np.dot(self.depth_inversa,puntos_uv1)# calcular cordenadas normalizadas
        puntos_xyz=z*puntos_xy1_n# calular cordenadas espaciales de la forma [X;Y;Z]. son vectores columna
        #devolver puntos a forma vector fila [(x,y,z)].separando los putos
        puntos_xyz=np.concatenate((puntos_xyz[0].reshape(-1,1),puntos_xyz[1].reshape(-1,1),puntos_xyz[2].reshape(-1,1)),axis=1)
        return (puntos_xyz) #entregar array de la forma shape(n,3)
    #encuentra la distancia euclidiana entre los puntos contiguos XYZ y los compara  con la distancia de construcción de los puntos del patron
    def calcular_distancia(self,puntos_xyz):
        distancias=[]
        error_distancias=[]
        for i in range(puntos_xyz.shape[0]-1):
            vector=puntos_xyz[i+1]-puntos_xyz[i]
            distancia=np.sqrt(np.sum(np.power(vector,2)))
            distancias.append(distancia)
        distancias=np.array(distancias).reshape(-1,1)/10
        error_distancias=np.absolute(self.distancia_patron-distancias)
        error_distancias=100*error_distancias/self.distancia_patron
        self.error_promedio=np.sum(error_distancias)/error_distancias.size
        return distancias,error_distancias,self.error_promedio
    #la distancia se calcula entre dos puntos contiguos
    #d=diametro de los circulos
    #nf=numero de filas del patron
    #nc=numero de circulos por fila
    #los puntos estan agrupados por filas, los puntos de cada fila se encuentran ordenados de izquierda a derecha y las filas de arriba hacia abajo
    def formar_distancia(self,d):
        nc=self.nc-1# al ser puntos medio entre circulos se pierde un punto por cada fila
        distancias_patron=np.ones((self.nf*nc-1,1))#al ser distancia sobre puntos contiguos la informacion disminuye en un dato
        #formar distancias teniendo en cuenta la estructura del patron
        for i in range(distancias_patron.size):
            if (i+1)%nc:# puntos contiguos de una misma fila
                distancias_patron[i]=2*d
            else:#puntos contiguos, punto extremo derecho fila superior, punto extremo izquierdo fila inferior
                if (i+1)%(2*nc):# por se patron asimetrico,las distancias entre extremos cambian de una fila a otra
                    distancias_patron[i]=np.sqrt((2*(nc-1)*d-d)**2+d**2)#fila impar
                else:
                    distancias_patron[i]=np.sqrt((2*(nc-1)*d+d)**2+d**2)#fila par
        return distancias_patron


In [ ]:
#from medicion import *
import sys# las sisguientes dos lineas ayudan a ubicar al archivo calibracion.py
sys.path.append("/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/")
from calibracion import *
path_configuracion="/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuraciónPrueba/"
nombre_configuracion="distorsionz"
path_img="/home/estufab4/Desktop/flujo de bagazo/codigo/img_medicion/longitud"
d=100
nf=8
nc=5
medir=longitud2(path_configuracion,nombre_configuracion,d,nf,nc)
ret,puntos_uv,z,img_puntos=medir.obtener_uvz(path_img,"gray","depth",9)

if ret:
        print("distancia minima",z.min())
        print("distancia maxima",z.max())
        print("distancia media",z.mean())
        xyz=medir.obtener_xyz(puntos_uv,z)
        distancias2,error_distancias2,error_promedio2=medir.calcular_distancia(xyz)
        for i in range(distancias.size):
            print("puntos {} y {} : las siguientes son unidades en milimetros".format(i,i+1))
            print("distancia calculada:{}".format(distancias2[i]))
            print("distancia original:{}".format(medir.distancia_patron[i]))
            print("error en porcentaje :{}%".format(error_distancias2[i]))
        print("error promedio en porcentaje :{}%".format(error_promedio2))
        graficar([img_puntos],1,1,["imagen depth con puntos detectados"])




In [ ]:
from tabulate import tabulate
tabla=[]
for distancia,distancia_patron,error,i in zip(distancias2,medir.distancia_patron,error_distancias2,range(distancias.size)):
    tabla.append(["{} y {}".format(i,i+1),distancia,distancia_patron,error])
print("error promedio",error_distancias2.mean())
print("Desviación error",error_distancias2.std())
print(tabulate(tabla,headers=["puntos","V medido[mm]","V Real[mm]","error%"]))
hist2,x2=np.histogram(error_distancias2,bins=20)
hist2=100*hist/len(error_distancias)
plt.hist(x00[:-1],x00, weights=hist00,color="r",label="Media: {:.2f}, STD= {:.2f}".format(error_distancias00.mean(),error_distancias00.std()))
plt.hist(x0[:-1],x0, weights=hist0,color="y",label="Media: {:.2f}, STD= {:.2f}".format(error_distancias0.mean(),error_distancias0.std()))

# plt.hist(x[:-1],x, weights=hist,color="y",label="Media: {:.2f}, STD= {:.2f}".format(error_distancias.mean(),error_distancias.std()))
plt.hist(x2[:-1],x2, weights=hist2,color="b",label="Media: {:.2f}, STD= {:.2f}".format(error_distancias2.mean(),error_distancias2.std()))
plt.title("Error longitud")
plt.grid()
plt.legend(loc='upper right')

# medición con calibración 2.0


# segmentado objeto negro fondo blanco
1. la funcion resive una imagen RGB y devuelve una imagen binaria. pixeles en uno pertenecen al objeto. pixeles en cero pertenecen al fondo.


In [ ]:

from skimage import morphology
import numpy as np
import cv2
import sys# las sisguientes dos lineas ayudan a ubicar al archivo calibracion.py
sys.path.append("/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/")
from calibracion import graficar
def segmetador(img,negro=True):
    n=300
    kerner=kernel = np.ones((n,n),np.uint8)
    img_gray=cv2.cvtColor(img.copy(), cv2.COLOR_RGB2GRAY)
    #img_= cv2.medianBlur(img_gray,15)
    if negro:
        criterio= cv2.THRESH_BINARY+cv2.THRESH_OTSU
        img_gray = cv2.morphologyEx(img_gray, cv2.MORPH_BLACKHAT, kernel)#MORPH_BLACKHAT
       
    else:
        criterio=cv2.THRESH_BINARY+cv2.THRESH_OTSU
    thr,mascara =cv2.threshold(img_gray,0,1,criterio)
    mascara.dtype=bool## remove_small solo funciona con variables binarias
    mascara=morphology.remove_small_objects(mascara,50,connectivity=10)
    mascara.dtype=np.uint8
    img_mascara=cv2.bitwise_and(img,img,mask=mascara)
    return mascara,img_mascara

# abrir img
img=cv2.imread("/home/estufab4/Desktop/flujo de bagazo/codigo/img_medicion/area/gray_2.png")
mascara,img_mascara=segmetador(img,1)

graficar([img,mascara,img_mascara],1,3,["original","mascara","original X mascara"])

# medición de area en cordenadas uv
1. la clase debe cargar el archivo de configuracin que contiene los datos de la calibración
2. la case debe funcionar independiente del segmentador
3. la clase tendra una función que resive un par de imagenes. RGB y profundidad y le quita la distorsión 
4. la clase tendra una función que resiva una imagen de profundidad y una mascara y apartir de ello calculara area.
5. la clase tendra una función que realizara el cambio de dominio de RGB al dominio de profundidad antes de  entrar la imagen a un segmentador.


# calibración 1.0

In [ ]:
from medicion import *
from tabulate import tabulate
class area(imagen):
    def __init__(self,path_configuracion,nombre_configuracion):
      
        imagen.__init__(self,path_configuracion,nombre_configuracion)
        self.fx=self.depth_intrinseca[0,0]
        self.fy=self.depth_intrinseca[1,1]
        self.constante_area=1/(self.fx*self.fy)
    def abrir_imgs(self,path, indicador1,indicador2,nombre):
        #abrir imagenes
        img2_depth=0
        img_depth=0
        ret,img_rgb,img_depth=self.abrir_par_img(path,indicador1,indicador2,nombre)
       
        if ret:
            # recuperar puntos perdidos
            img_depth=self.recuperar(img_depth)
            #quitar distorsion z
            img_depth=self.distorsion_z(img_depth,filtrar=False)
            #quitar distorsion
            img_rgb,img_depth=self.distorsion(img_rgb,img_depth)
            # cambiar de dominio
            img2_depth=self.rgb2_depth(img_rgb)
        return ret,img2_depth,img_depth
    def calcular_area(self,img,mascara):
        img_z=cv2.bitwise_and(img,img,mask=mascara)
        img_z=np.array(img_z/10,dtype=np.float64)
        #print("distancia maxima de profundidad en mm: {}".format(img_z.max()))
        img_z2=img_z*img_z
        img_area=self.constante_area*img_z2
        area=np.sum(img_area)
        return area
    def calcular_error(self,valor_real,valor_medido):
        e=100*(valor_real-valor_medido)/valor_real
        return e
    def recuperar(self,img):
        mask=img.copy()
        mask[mask<400]=1
        mask[mask!=1]=0
        mask=mask.astype(np.uint8)
        dst = cv2.inpaint(img,mask,1,cv2.INPAINT_TELEA)
#         dst=cv2.blur(dst,(3,3))
        return dst

# calibración 1a
1. mejora de la información al implementar recuerapación
2. mejora de la infomración con respecto a la distorsión de profundidad


In [ ]:
from medicion import *
from tabulate import tabulate
class areaa(imagen):
    def __init__(self,path_configuracion,nombre_configuracion):
      
        imagen.__init__(self,path_configuracion,nombre_configuracion,pendientes2=True)
        self.fx=self.depth_intrinseca[0,0]
        self.fy=self.depth_intrinseca[1,1]
        self.constante_area=1/(self.fx*self.fy)
    def abrir_imgs(self,path, indicador1,indicador2,nombre):
        #abrir imagenes
        img2_depth=0
        img_depth=0
        ret,img_rgb,img_depth=self.abrir_par_img(path,indicador1,indicador2,nombre)
       
        if ret:
            # recuperar puntos perdidos
            img_depth=self.recuperar(img_depth)
            #quitar distorsion z
            img_depth=self.distorsion_z2(img_depth,filtrar=False)
            #quitar distorsion
            img_rgb,img_depth=self.distorsion(img_rgb,img_depth)
            # cambiar de dominio
            img2_depth=self.rgb2_depth(img_rgb)
        return ret,img2_depth,img_depth
    def calcular_area(self,img,mascara):
        img_z=cv2.bitwise_and(img,img,mask=mascara)
        img_z=np.array(img_z/10,dtype=np.float64)
        #print("distancia maxima de profundidad en mm: {}".format(img_z.max()))
        img_z2=img_z*img_z
        img_area=self.constante_area*img_z2
        area=np.sum(img_area)
        return area
    def calcular_error(self,valor_real,valor_medido):
        e=100*(valor_real-valor_medido)/valor_real
        return e
    def recuperar(self,img):
        mask=img.copy()
        mask[mask<400]=1
        mask[mask!=1]=0
        mask=mask.astype(np.uint8)
        dst = cv2.inpaint(img,mask,1,cv2.INPAINT_TELEA)
#         dst=cv2.blur(dst,(3,3))
        return dst

# calibración 2.0

In [ ]:
from medicion import *
from tabulate import tabulate
import numpy as np
class area2(imagen):
    def __init__(self,path_configuracion,nombre_configuracion):
      
        imagen.__init__(self,path_configuracion,nombre_configuracion)
        self.calcular=np.poly1d(self.polinomio)
    def abrir_imgs(self,path, indicador1,indicador2,nombre):
        #abrir imagenes
        img2_depth=0
        img_depth=0
        ret,img_rgb,img_depth=self.abrir_par_img(path,indicador1,indicador2,nombre)
       
        if ret:
            #quitar distorsion z
            img_depth=self.distorsion_z(img_depth,filtrar=True)
            #quitar distorsion
            img_rgb,img_depth=self.distorsion(img_rgb,img_depth)
            # cambiar de dominio
            img2_depth=self.rgb2_depth(img_rgb)
        return ret,img2_depth,img_depth
    def calcular_area(self,img,mascara):
        img_z=cv2.bitwise_and(img,img,mask=mascara)
        img_z=np.array(img_z/10,dtype=np.float64)
        img_z=img_z[img_z>0]
       
        #print("distancia maxima de profundidad en mm: {}".format(img_z.max()))
        img_area=self.calcular(img_z)
        area=np.sum(img_area)
        return area
    def calcular_error(self,valor_real,valor_medido):
        e=100*(valor_real-valor_medido)/valor_real
        return e

# calibración 2a

In [ ]:
from medicion import *
from tabulate import tabulate
import numpy as np
class area2a(imagen):
    def __init__(self,path_configuracion,nombre_configuracion):
      
        imagen.__init__(self,path_configuracion,nombre_configuracion,pendientes2=True)
        p=np.array([ 2.90651860e-06,  2.62350174e-04, -1.07560053e-01])
        self.calcular=np.poly1d(p)
    def abrir_imgs(self,path, indicador1,indicador2,nombre):
        #abrir imagenes
        img2_depth=0
        img_depth=0
        ret,img_rgb,img_depth=self.abrir_par_img(path,indicador1,indicador2,nombre)
       
        if ret:
            #quitar distorsion z
            img_depth=self.distorsion_z2(img_depth,filtrar=False)
            #quitar distorsion
            img_rgb,img_depth=self.distorsion(img_rgb,img_depth)
            # cambiar de dominio
            img2_depth=self.rgb2_depth(img_rgb)
        return ret,img2_depth,img_depth
    def calcular_area(self,img,mascara):
        img_z=cv2.bitwise_and(img,img,mask=mascara)
        img_z=np.array(img_z/10,dtype=np.float64)
        img_z=img_z[img_z>0]
       
        #print("distancia maxima de profundidad en mm: {}".format(img_z.max()))
        img_area=self.calcular(img_z)
        area=np.sum(img_area)
        return area
    def calcular_error(self,valor_real,valor_medido):
        e=100*(valor_real-valor_medido)/valor_real
        return e

# calibración 3.0

In [ ]:
from medicion import *
from tabulate import tabulate
import numpy as np
from keras.models import Sequential,model_from_json
from keras.layers import Dense, Activation
from keras.callbacks import ModelCheckpoint
import random
import matplotlib.pyplot as plt
class area3(imagen):
    def __init__(self,path_configuracion,nombre_configuracion):
        imagen.__init__(self,path_configuracion,nombre_configuracion)
#         self.calcular=np.poly1d(self.polinomio)
        self.maxi=2500
        # cargar modelo
        json_file = open("{}net.json".format(path_configuracion), 'r')
        loaded_model_json = json_file.read()
        json_file.close()
        loaded_model = model_from_json(loaded_model_json)
#         se cargan los pesos (weights) en el nuevo modelo
        loaded_model.load_weights("{}net.h5".format(path_configuracion))
        self.model=loaded_model
    
    def abrir_imgs(self,path, indicador1,indicador2,nombre):
        #abrir imagenes
        img2_depth=0
        img_depth=0
        ret,img_rgb,img_depth=self.abrir_par_img(path,indicador1,indicador2,nombre)
       
        if ret:
            #quitar distorsion z
            img_depth=self.distorsion_z(img_depth,filtrar=True)
            #quitar distorsion uv para ambas imagesnes
            img_rgb,img_depth=self.distorsion(img_rgb,img_depth)
            # llevar la información del dominio RGB al dominio de profundidad
            img2_depth=self.rgb2_depth(img_rgb)
        return ret,img2_depth,img_depth
    def calcular_area(self,img,mascara):
        img_z=np.array(cv2.bitwise_and(img,img,mask=mascara),dtype=np.float64)/10
        img_z=img_z[img_z>0].reshape(-1,1)
        img_z2=np.power(img_z,2)
        img_z2=np.concatenate([img_z,img_z2],axis=1)
        #print("distancia maxima de profundidad en mm: {}".format(img_z.max()))
        img_area=self.model.predict(self.normalizar(img_z2))
        area=np.sum(img_area)
        return area
    def calcular_error(self,valor_real,valor_medido):
        e=100*(valor_real-valor_medido)/valor_real
        return e
    def normalizar(self,IN,entrenar=False):
        
        IN[:,0]=IN[:,0]/self.maxi
        IN[:,1]=IN[:,1]/np.power(self.maxi,2)
        return IN

# calibración 3.a

In [ ]:
#
from medicion import *
from tabulate import tabulate
import numpy as np
from keras.models import Sequential,model_from_json
from keras.layers import Dense, Activation
from keras.callbacks import ModelCheckpoint
import random
import matplotlib.pyplot as plt
class area3a(imagen):
    def __init__(self,path_configuracion,nombre_configuracion):
        imagen.__init__(self,path_configuracion,nombre_configuracion)

        # cargar modelo
        json_file = open("{}net.json".format(path_configuracion), 'r')
        loaded_model_json = json_file.read()
        json_file.close()
        loaded_model = model_from_json(loaded_model_json)
#         se cargan los pesos (weights) en el nuevo modelo
        loaded_model.load_weights("{}net.h5".format(path_configuracion))
        self.model=loaded_model
        
        self.fvo=self.depth_intrinseca[0,2]
        self.fuo=self.depth_intrinseca[1,2]
        self.shape=[480,640]
        self.fvo=self.depth_intrinseca[0,2]
        self.fuo=self.depth_intrinseca[1,2]
        self.U,self.V=self.calcularUV(self.shape)
        self.mini=300**2# Secuencia de números enteros sin duplicados
        self.maxi=2500**2
    def calcularUV(self,shape):
        v,u=np.mgrid[0:self.shape[0],0:self.shape[1]]
        u=np.array(u-self.fuo,dtype=np.float32)
        v=np.array(v-self.fvo,dtype=np.float32)
        u=u**2# distancias al cuadrado
        v=v**2
        u=(u-u.min())/(u.max()-u.min())
        v=(v-v.min())/(v.max()-v.min())
#         radio=np.sqrt(np.power(u,2)+np.power(v,2))
#         radio=(radio-radio.min())/(radio.max()-radio.min())
        return u,v
    
    def abrir_imgs(self,path, indicador1,indicador2,nombre):
        #abrir imagenes
        img2_depth=0
        img_depth=0
        ret,img_rgb,img_depth=self.abrir_par_img(path,indicador1,indicador2,nombre)
       
        if ret:
            #quitar distorsion z
            img_depth=self.distorsion_z(img_depth,filtrar=True)
            #quitar distorsion uv para ambas imagesnes
            img_rgb,img_depth=self.distorsion(img_rgb,img_depth)
            # llevar la información del dominio RGB al dominio de profundidad
            img2_depth=self.rgb2_depth(img_rgb)
        return ret,img2_depth,img_depth
    def calcular_area(self,img,mascara):
        img_z=img[mascara>0].reshape(-1)/10
        img_z2=self.normalizar(img_z).reshape(-1,1)
        u=self.U[mascara>0].reshape(-1,1)
        v=self.V[mascara>0].reshape(-1,1)
        IN=np.concatenate([img_z2,u,v],axis=1)
        #print("distancia maxima de profundidad en mm: {}".format(img_z.max()))
        img_area=self.model.predict(IN)
        area=np.sum(img_area)
        return area
    def calcular_error(self,valor_real,valor_medido):
        e=100*(valor_real-valor_medido)/valor_real
        return e
    def normalizar(self,zpx):
        
        zpx=zpx**2
        print(zpx.shape)
        zpx=(zpx-self.mini)/(self.maxi-self.mini)
        return zpx
    def desnormalizar(self,zpx):
        zpx=np.sqrt(zpx*(self.maxi-self.mini)+self.mini)
        return zpx

# area 4

In [ ]:
from medicion import *
from tabulate import tabulate
class area4(imagen):
    def __init__(self,path_configuracion,nombre_configuracion):
      
        imagen.__init__(self,path_configuracion,nombre_configuracion,pendientes2=1)
        self.fx=self.depth_intrinseca[0,0]
        self.fy=self.depth_intrinseca[1,1]
        self.fvo=self.depth_intrinseca[0,2]
        self.fuo=self.depth_intrinseca[1,2]
        self.constante_area=1/(self.fx*self.fy)
        self.beta=np.array([ 5.83234140e-02,  9.53776679e-02,  3.02404506e-02, -5.51787924e+06,-1.65675186e+06])
        self.minZ=10000
        self.maxZ=25000
        self.U=[]
        self.V=[]
    def normalizarz(self, z):
        z=(z-self.minZ)/(self.maxZ-self.minZ)
        return z
    def normalizarUV(self,x):# recive el vector u o el vector v, solo uno a la ves
        x=(x-x.mean())/(x.std())
        return x
    def calcularUV(self,shape):
        v,u=np.mgrid[0:shape[0],0:shape[1]]
        self.U=np.array(u-self.fuo,dtype=np.float64)
        self.V=np.array(v-self.fvo,dtype=np.float64)
        self.U=self.normalizarUV(self.U)
        self.V=self.normalizarUV(self.V)
    def abrir_imgs(self,path, indicador1,indicador2,nombre):
        #abrir imagenes
        img2_depth=0
        img_depth=0
        ret,img_rgb,img_depth=self.abrir_par_img(path,indicador1,indicador2,nombre)
       
        if ret:
            # recuperar puntos perdidos
            img_depth=self.recuperar(img_depth)
            #quitar distorsion z
            img_depth=self.distorsion_z2(img_depth,filtrar=False)
            #quitar distorsion
            img_rgb,img_depth=self.distorsion(img_rgb,img_depth)
            # cambiar de dominio
            img2_depth=self.rgb2_depth(img_rgb)
        return ret,img2_depth,img_depth
    def funcionModeloGaus(self,listZ,listU,listv,b):
        
        # xi,ui,vi, son matrices de nx1
        # list son listas de nxm: m se refiere al numero de imagenes utilizadas,n el numero de px que contiene el objeto en la imagen. solo se guardaron px correspondientes al objeto
        #list son listas cada elemento es una matriz de nx1, sin embargo n puede variar de elemento en elemento, es por eso que se utiliza listas y no matrices y que pueden variar los elementos
        y=np.array([(b[0]*zi**2+(zi*b[1])*np.exp(-((ui)/b[3]+(vi)/b[4]))+b[2]).sum() for zi, ui, vi,  in zip(listZ, listU, listv)])
        return y#
    def calcular_area(self,img,mascara):
        if self.U==[]:
            shape=img.shape
            self.calcularUV(shape)
        indexMascara=np.where(mascara==1)   
        
        z_mascara=[self.normalizarz(img.astype(np.float64)[indexMascara].reshape(-1,1))]
        u_mascara=[self.U[indexMascara].reshape(-1,1)]
        v_mascara=[self.V[indexMascara].reshape(-1,1)]
        area=self.funcionModeloGaus(z_mascara,u_mascara,v_mascara,self.beta)[0]
        

        return area
    def calcular_error(self,valor_real,valor_medido):
        e=100*(valor_real-valor_medido)/valor_real
        return e
    def recuperar(self,img):
        mask=img.copy()
        mask[mask<400]=1
        mask[mask!=1]=0
        mask=mask.astype(np.uint8)
        dst = cv2.inpaint(img,mask,1,cv2.INPAINT_TELEA)
#         dst=cv2.blur(dst,(3,3))
        return dst

In [ ]:

from tabulate import tabulate
import numpy as np
def medir(path_config,list_config,path_img,valor_real,titulo="Error"):
    list_titulos=["error porcentual"]
    list_tablas=[]
    ejex=[]
    ejey=[]
    for config in list_config:
        print(config)
        a=area2a(path_config,config)

            # abrir el par de imagenes
        list_medidas=[]
        list_error=[]
        list_media=[]
        for i in range(1,100):
            ret,img2_depth,img_depth=a.abrir_imgs(path=path_img,
                                            indicador1="gray",
                                            indicador2="depth",
                                            nombre=i
                                           )
            if ret:
                #segmentar
                
                mascara,_=segmetador(img2_depth)
                
                #visualizar 
#                 graficar([img_depth,img2_depth,mascara],1,3,["img original profundidad","img rgb en profundidad","mascara"])

                area_calculada=a.calcular_area(img_depth,mascara)
                
                #valor_real=10000
                error=a.calcular_error(valor_real,area_calculada)
                list_medidas.append(area_calculada)
                list_error.append(error)
                list_media.append(img_depth.mean()/10)
                print("valor medio profundidad mm: ",img_depth.mean()/10)
                print("valor real mm²: {}".format(valor_real))
                print("area calculada mm²:{}".format(area_calculada))
    #             print("porcentaje de error:{}".format(error))
    #     print(valor_real)
        tabla=zip(list_medidas,list_error,list_media)
        ejex.append(list_media)
        ejey.append(np.absolute(list_error))
        list_titulos.append(config)
        print("valor real:{}".format(valor_real))
#         print(tabulate(tabla,headers=["v calculado","error","distancia media"]))
        print(" Error absoluto medio: ", np.absolute(np.array(list_error)).mean())
        plot([list_media],[np.absolute(list_error)],1,1,[titulo],[["Profundidad","Error absoluto %"]],tipo=1)
        tabla=np.array([[x,y] for x,y in zip(list_media,np.absolute(list_error)) if x>=1000 and x <2400 ])
#         return list_error,list_media
        return tabla[:,1],tabla[:,0]

In [ ]:
# medición circulos
path_img="/home/estufab4/Desktop/flujo de bagazo/codigo/img_medicion/circulos"
path_config="/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuraciónPrueba/"
# path_config="/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuraciónPrueba/"
# path_config= "/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuracion_union_150/"
#             /home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuracion_union_120
    
list_config=["distorsionz"]
valor_real=((150.0**2)*np.pi)/4
circulos,list_media_circulos=medir(path_config,list_config,path_img,valor_real,titulo="Circulos")
plt.xlim=((1000,1500))
plt.ylim=((0,1.5))

tabla=np.array([[x,y] for x,y in zip(list_media_circulos,np.absolute(circulos)) if x>=1000 and x <2400 ])
print("numero de medidas: {}".format(len(circulos)))
print("error absoluto medio: ",tabla[:,1].mean())
print("error absoluto maximo: ",tabla[:,1].max())
print("desviación error absoluto: ",tabla[:,1].std())
plt.plot(tabla[:,0],tabla[:,1])
plt.show()


In [ ]:
# print("Antes Media: {:.2f},Desviación: {:.2f}".format(circulos0.mean(),circulos0.std()))
print("Después Media: {:.2f},Desviación: {:.2f}".format(circulos.mean(),circulos.std()))
ax=plt.hist(circulos,color="b")
# ax=plt.hist(circulos0,color="r")
plt.show()
# plt.plot(list_media_circulos,circulos0,label="Antes",color="r")
plt.plot(list_media_circulos,circulos,label="Después",color="b")
plt.legend(loc='upper center', shadow=True, fontsize='x-large')
plt.show()

In [ ]:
list_media_circulos

In [ ]:
# medición poligono
path_img="/home/estufab4/Desktop/flujo de bagazo/codigo/img_medicion/poligono"
# path_config="/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuracion_union_150/"
# list_config=["configuracion_d5_90"]
valor_real=146.1418
poligono,list_media_poligono=medir(path_config,list_config,path_img,valor_real,titulo="Poligono")
print("numero de medidas: {}".format(len(poligono)))

In [ ]:
# medición Piñon 
path_img="/home/estufab4/Desktop/flujo de bagazo/codigo/img_medicion/piñon/"
# path_config="/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuracion_union_150/"
# list_config=["configuracion_d5_90"]
valor_real=155.24847
piñon,list_media_piñon=medir(path_config,list_config,path_img,valor_real,titulo="piñon")
print("numero de medidas: {}".format(len(piñon)))

In [ ]:
total0=np.absolute(np.concatenate([circulos,poligono,piñon]))
hist,x=np.histogram(total0,bins=20,range=(0,6))
hist=100*hist/len(total0)
plt.hist(x[:-1],x, weights=hist)
plt.title("Media: {:.2f}, STD= {:.2f}".format(total0.mean(),total0.std()))
plt.grid()
print("error promedio total:",total0.mean())
print("error total:",total0.sum())
plot([list_media_circulos,list_media_poligono,list_media_piñon],[np.absolute(circulos),np.absolute(poligono),np.absolute(piñon)],list_titulos=[" Error de área","Circulo","Polígono","piñón"],ejes=[["Profunidad mm","error absoluto %"]],tipo=1)


In [ ]:
total=np.absolute(np.concatenate([circulos,poligono,piñon]))
hist,x=np.histogram(total,bins=20,range=(0,6))
hist=100*hist/len(total)
plt.hist(x[:-1],x, weights=hist)
plt.title("Media: {:.2f}, STD= {:.2f}".format(total.mean(),total.std()))
plt.grid()
print("error promedio total:",total.mean())
print("error total:",total.sum())
print(len(total))
plot([list_media_circulos,list_media_poligono,list_media_piñon],[np.absolute(circulos),np.absolute(poligono),np.absolute(piñon)],list_titulos=[" Error de área","Circulo","Polígono","piñón"],ejes=[["Profunidad mm","error absoluto %"]],tipo=1)


In [ ]:
hist,x=np.histogram(total,bins=20,range=(0,6))
hist=100*hist/len(total)
plt.hist(x[:-1],x, weights=hist,color="b",label="Media: {:.2f}; STD: {:.2f}".format(total.mean(),total.std()))
print(len(total0),(len(total)))
hist,x=np.histogram(total0,bins=20,range=(0,6))
hist=100*hist/len(total0)
plt.hist(x[:-1],x, weights=hist,color="r",label="Media: {:.2f}; STD: {:.2f}".format(total0.mean(),total0.std()))
plt.title("Error Area")
plt.grid()
plt.legend(loc='upper right')

In [ ]:
# medición cartulina
path_img="/home/estufab4/Desktop/flujo de bagazo/codigo/img_medicion/cartulina/"
path_config="/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuracion_union_150/"
list_config=["configuracion_d5_90"]
valor_real=322284
cartulina,list_media_cartulina=medir(path_config,list_config,path_img,valor_real,titulo="cartulina")
tabla=np.array([[x,y] for x,y in zip(list_media_cartulina,np.absolute(cartulina)) if x>=1500 and x <2200 ])
print("error absoluto medio: ",tabla[:,1].mean())
print("error absoluto maximo: ",tabla[:,1].max())
print("desviación error absoluto: ",tabla[:,1].std())
plt.plot(tabla[:,0],tabla[:,1])
plt.show()

# Calcular volumen
1. solo podra calcular volumen de formamas limitadas. Esto es debido a que solo se tiene la vista superior del objeto para poder calcularla.
2. dado un objeto, denotaremos como cara superior, la superficie que mira directamente la camara. se denotara como cara inferior la superficie atras del objeto, opuesta a la carmara.
3. el sismeta medira con mayor presicion toda aquella inforamcion que este dentro un marco a 60 px por cada lado.
4. la parte inferior del objeto debe estar limitada por una superficie referencia, dicha información de la supeficie debe ser tomada o recontruida antes de poder empezar a calcular volumen,el sistema medirá con mayor presición si la superficie y la camara forman paralelas
5. si denotamos como la profundidad la distancia entre la superficie referencia y la camara, y dividimos al objeto en areas transversales, perpendiculares a z, el sistema solo podra calcular el volumen a objetos cuyas areas de la cara superior sean menores o iguales a las de la cara inferior.
![title](volumen.png)
6. en la anterior figura se mutra de forma grafica como se cacula el volumen. como se puede ver la  cara superior puede tener cualquier nivel o curva,siempre y cuando no sea mayor a la cara inferior. tambien cabe destacar como la parte inferior esta cortada por un plano.
# clase
1. la clase 

In [ ]:
from medicion import imagen
import sys# las sisguientes dos lineas ayudan a ubicar al archivo calibracion.py
sys.path.append("/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion")
from calibracion import *
import  os
class volumen(imagen):
    def __init__(self,path_configuracion,nombre_configuracion):
      
        imagen.__init__(self,path_configuracion,nombre_configuracion)
        self.fx=self.depth_intrinseca[0,0]
        self.fy=self.depth_intrinseca[1,1]
        self.constante_area=1/(self.fx*self.fy)
       
 
    def preprocesar_imgs(self,img_rgb,img_depth,f_segmentador):
        #quitar distorsion z
        img_depth=self.distorsion_z(img_depth,filtrar=True)
     
        #quitar distorsion uv
        img_rgb,img_depth=self.distorsion(img_rgb,img_depth)
        # cambiar de dominio rgb a depth
        img2_depth=self.rgb2_depth(img_rgb)
        # encontrar mascara
        ## hay que corregir 
        mascara,img=f_segmentador(img2_depth)
        return img_depth,mascara
    def calcular_volumen(self,img,mascara,img_ref):
       
        img_z=cv2.bitwise_and(img,img,mask=mascara)
        img_ref=cv2.bitwise_and(img_ref,img_ref,mask=mascara)

        img_z=np.array(img_z,dtype=np.float64)

        objeto=img_z.copy()[img_z>0].reshape(-1)
        ref=img_ref.copy()[img_ref>0].reshape(-1)
        i_objeto=range(objeto.size)
        i_ref=range(ref.size)
        #plot(list_x=[i_objeto,i_ref],list_y=[objeto,ref],filas=1,colum=2,list_titulos=["altura objeto","altura referencia"],ejes=[["px index","profundidad"],["px index","profundidad"]])
        img_z2=img_z.copy()*img_z.copy()
        
        img_area=self.constante_area*img_z2

        img_altura=img_ref.copy()-img_z.copy()
        
        img_volumen=img_area*img_altura
        volumen=np.sum(img_volumen)
        return volumen
    def calcular_error(self,valor_real,valor_medido):
        e=100*(valor_real-valor_medido)/valor_real
        return e
    def mejorar_referencia(self,img_ref):

        img_ref[img_ref==0]=img_ref.mean()
        img_ref=self.distorsion_z(img_ref,1)
        return img_ref
    def abrir_img_ref(self,path,nombre):
        path_ref=os.path.join(path, '{}.png'.format(nombre))
      
        ret=cv2.haveImageReader(path_ref)
        img_ref=0
        if ret:
            img_ref=cv2.imread(path_ref,cv2.IMREAD_UNCHANGED)[:,:,0]
            img_ref=self.mejorar_referencia(img_ref)
        return ret,img_ref
    def recuperar(self,img):
        mask=img.copy()
        mask[mask<400]=1
        mask[mask!=1]=0
        mask=mask.astype(np.uint8)
        dst = cv2.inpaint(img,mask,1,cv2.INPAINT_TELEA)
        graficar([dst,img],1,2)
        dst=cv2.blur(dst,(3,3))
        return dst

# medición con recuperación de la información 

In [ ]:
# calibración incluyendo la recuperación de la información 
from medicion import imagen
import sys# las sisguientes dos lineas ayudan a ubicar al archivo calibracion.py
sys.path.append("/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion")
from calibracion import *
import  os
class volumena(imagen):
    def __init__(self,path_configuracion,nombre_configuracion):
      
        imagen.__init__(self,path_configuracion,nombre_configuracion)
        self.fx=self.depth_intrinseca[0,0]
        self.fy=self.depth_intrinseca[1,1]
        self.constante_area=1/(self.fx*self.fy)
       
 
    def preprocesar_imgs(self,img_rgb,img_depth,f_segmentador):
        buffer=img_depth.copy()
        #recuperar información
        img_depth=self.recuperar(img_depth)
        #quitar distorsion z
        img_depth=self.distorsion_z(img_depth,filtrar=True)
        graficar([img_depth,buffer],1,2)
        #quitar distorsion uv
        img_rgb,img_depth=self.distorsion(img_rgb,img_depth)
        # cambiar de dominio rgb a depth
        img2_depth=self.rgb2_depth(img_rgb)
        # encontrar mascara
        ## hay que corregir 
        mascara,img=f_segmentador(img2_depth)
        return img_depth,mascara
    def calcular_volumen(self,img,mascara,img_ref):
       
        img_z=cv2.bitwise_and(img,img,mask=mascara)
        img_ref=cv2.bitwise_and(img_ref,img_ref,mask=mascara)

        img_z=np.array(img_z,dtype=np.float64)

        objeto=img_z.copy()[img_z>0].reshape(-1)
        ref=img_ref.copy()[img_ref>0].reshape(-1)
        i_objeto=range(objeto.size)
        i_ref=range(ref.size)
        #plot(list_x=[i_objeto,i_ref],list_y=[objeto,ref],filas=1,colum=2,list_titulos=["altura objeto","altura referencia"],ejes=[["px index","profundidad"],["px index","profundidad"]])
        img_z2=img_z.copy()*img_z.copy()
        
        img_area=self.constante_area*img_z2

        img_altura=img_ref.copy()-img_z.copy()
        
        img_volumen=img_area*img_altura
        volumen=np.sum(img_volumen)
        index=np.where(mascara==1)
        print(img_area.sum(),"area")
        
        print(img_ref[index].mean(),"ref")
        print(img_z[index].mean(),"dep")
        print(img_altura[index].mean(),"altura")
        print(volumen,"vol")
        img_ref[img_ref==0]=img_ref[index].mean()-img_ref[index].mean()*0.01
        img_z[img_z==0]=img_z[index].mean()-img_z[index].mean()*0.01
        img_altura[img_altura==0]=img_altura[index].mean()-img_altura[index].mean()*0.01
        img_volumen[img_volumen==0]=img_volumen[index].mean()-img_volumen[index].mean()*0.01
        
        
        
        graficar2([img_ref,img_z],1,2,["ref","z"])
        graficar2([img_altura,img_volumen],1,2,["altura","vol"])
        return volumen
    def calcular_error(self,valor_real,valor_medido):
        e=100*(valor_real-valor_medido)/valor_real
        return e
    def mejorar_referencia(self,img_ref):

#         img_ref=self.recuperar(img_ref)
        img_ref=self.distorsion_z(img_ref,1)
        return img_ref
    def abrir_img_ref(self,path,nombre):
        path_ref=os.path.join(path, '{}.png'.format(nombre))
      
        ret=cv2.haveImageReader(path_ref)
        img_ref=0
        if ret:
            img_ref=cv2.imread(path_ref,cv2.IMREAD_UNCHANGED)[:,:,0]
            img_ref=self.mejorar_referencia(img_ref)
        return ret,img_ref
    def recuperar(self,img):
        mask=img.copy()
        mask[mask<=300]=1
        mask[mask!=1]=0
        mask=mask.astype(np.uint8)
        dst = cv2.inpaint(img,mask,1,cv2.INPAINT_TELEA)
        dst=cv2.blur(dst,(3,3))
#         graficar([dst],1,1)
        return dst

# Medción de volumen 1.b
1. mejora en la distorsión de profundidad

In [ ]:
# calibración incluyendo la recuperación de la información 
from medicion import imagen
import sys# las sisguientes dos lineas ayudan a ubicar al archivo calibracion.py
sys.path.append("/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion")
from calibracion import *
import  os
class volumenb(imagen):
    def __init__(self,path_configuracion,nombre_configuracion):
      
        imagen.__init__(self,path_configuracion,nombre_configuracion,pendientes2=True)
        self.fx=self.depth_intrinseca[0,0]
        self.fy=self.depth_intrinseca[1,1]
        self.constante_area=1/(self.fx*self.fy)
       
 
    def preprocesar_imgs(self,img_rgb,img_depth,f_segmentador):
        buffer=img_depth.copy()
        #recuperar información
        img_depth=self.recuperar(img_depth)
        #quitar distorsion z
        img_depth=self.distorsion_z2(img_depth,filtrar=False)
#         graficar([img_depth,buffer],1,2)
        #quitar distorsion uv
        img_rgb,img_depth=self.distorsion(img_rgb,img_depth)
        # cambiar de dominio rgb a depth
        img2_depth=self.rgb2_depth(img_rgb)
        # encontrar mascara
        ## hay que corregir 
        mascara,img=f_segmentador(img2_depth)
        return img_depth,mascara
    def calcular_volumen(self,img,mascara,img_ref):
       
        img_z=cv2.bitwise_and(img,img,mask=mascara)
        img_ref=cv2.bitwise_and(img_ref,img_ref,mask=mascara)

        img_z=np.array(img_z,dtype=np.float64)

        objeto=img_z.copy()[img_z>0].reshape(-1)
        ref=img_ref.copy()[img_ref>0].reshape(-1)
        i_objeto=range(objeto.size)
        i_ref=range(ref.size)
        #plot(list_x=[i_objeto,i_ref],list_y=[objeto,ref],filas=1,colum=2,list_titulos=["altura objeto","altura referencia"],ejes=[["px index","profundidad"],["px index","profundidad"]])
        
        index=np.where(mascara==1)
        img_altura=img_ref.copy()-img_z.copy()
#         img_z=self.distorsion_z2(img_z,0)
        img_z2=img_z.copy()*img_z.copy()
        img_area=self.constante_area*img_z2

        img_altura=img_ref.copy()-img_z.copy()
        
        
        
        img_volumen=img_area*img_altura
        
        volumen=np.sum(img_volumen)
        print(img_area.sum(),"area")
        
        print(img_ref[index].mean(),"ref")
        print(img_z[index].mean(),"dep")
        print(img_altura[index].mean(),"altura")
        print(volumen,"vol")
        img_ref[img_ref==0]=img_ref[index].mean()-img_ref[index].mean()*0.01
        img_z[img_z==0]=img_z[index].mean()-img_z[index].mean()*0.01
        img_altura[img_altura==0]=img_altura[index].mean()-img_altura[index].mean()*0.01
        img_volumen[img_volumen==0]=img_volumen[index].mean()-img_volumen[index].mean()*0.01
        
        
        
        graficar2([img_ref,img_z],1,2,["ref","z"])
        graficar2([img_altura,img_volumen],1,2,["altura","vol"])
        volumen=volumen +0.09*volumen
        return volumen
    def calcular_error(self,valor_real,valor_medido):
        e=100*(valor_real-valor_medido)/valor_real
        return e
    def mejorar_referencia(self,img_ref):

        img_ref=self.recuperar(img_ref)
        img_ref=self.distorsion_z2(img_ref,1)
        return img_ref
    def abrir_img_ref(self,path,nombre):
        path_ref=os.path.join(path, '{}.png'.format(nombre))
      
        ret=cv2.haveImageReader(path_ref)
        img_ref=0
        if ret:
            img_ref=cv2.imread(path_ref,cv2.IMREAD_UNCHANGED)[:,:,0]
            img_ref=self.mejorar_referencia(img_ref)
        return ret,img_ref
    def recuperar(self,img):
        mask=img.copy()
        mask[mask<=3000]=1
        mask[mask!=1]=0
        mask=mask.astype(np.uint8)
        dst = cv2.inpaint(img,mask,1,cv2.INPAINT_TELEA)
        dst=cv2.blur(dst,(3,3))
#         graficar([dst],1,1)
        return dst

# calibración 2.0


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
class volumen2(imagen):
    def __init__(self,path_configuracion,nombre_configuracion):
      
        imagen.__init__(self,path_configuracion,nombre_configuracion)
        self.fx=self.depth_intrinseca[0,0]
        self.fy=self.depth_intrinseca[1,1]
        self.constante_area=1/(self.fx*self.fy)
        self.f_area=np.poly1d(self.polinomio)
 
    def preprocesar_imgs(self,img_rgb,img_depth,f_segmentador,negro=True):
        buffer=img_depth.copy()
        #quitar distorsion z
        img_depth=self.distorsion_z(img_depth,filtrar=True)
        graficar([img_depth,buffer])
        #quitar distorsion uv
#         img_rgb,img_depth=self.distorsion(img_rgb,img_depth)
        # cambiar de dominio rgb a depth
        img2_depth=self.rgb2_depth(img_rgb)
        # encontrar mascara
        ## hay que corregir 
        mascara,_=f_segmentador(img2_depth)
        return img_depth,mascara
    def calcular_volumen(self,img,mascara,img_ref):
      
        img_z=img.reshape(-1)
        mascara=mascara.reshape(-1)
        img_ref=img_ref.reshape(-1)
        # necesario para compatibilidad de dimensiones
        # extraer los pixeles del objetivo
        img_z=np.array([zpx for zpx,mpx in zip(img_z,mascara) if mpx==1])
        img_ref=np.array([rpx for rpx,mpx in zip(img_ref,mascara) if mpx==1])
        print(img_z.shape,img_ref.shape)
        img_z=np.array(img_z,dtype=np.float64)
#         graficar([img_z,img_ref],1,2,["img x mascara","ref"])

        img_area=self.f_area(img_z.copy())
        
        
        img_altura=img_ref-img_z
        img_altura=img_altura
        
        img_volumen=img_area*img_altura
        volumen=np.sum(img_volumen)
        return volumen
    def calcular_error(self,valor_real,valor_medido):
        e=100*(valor_real-valor_medido)/valor_real
        return e
    def mejorar_referencia(self,img_ref):

#         img_ref[img_ref==0]=img_ref.mean()
        img_ref=self.distorsion_z(img_ref,1)

        return img_ref
    def abrir_img_ref(self,path,nombre):
        path_ref=os.path.join(path, '{}.png'.format(nombre))
        print(path_ref)
        ret=cv2.haveImageReader(path_ref)
        img_ref=0
        if ret:
            img_ref=cv2.imread(path_ref,cv2.IMREAD_UNCHANGED)[:,:,0]
            img_ref=self.mejorar_referencia(img_ref)
        return ret,img_ref

# calibración 3.0

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from medicion import *
from keras.models import Sequential,model_from_json
from keras.layers import Dense, Activation
from keras.callbacks import ModelCheckpoint
import random
from ttictoc import tic,toc
from ttictoc import tic,toc
class volumen3(imagen):
    def __init__(self,path_configuracion,nombre_configuracion):
      
        imagen.__init__(self,path_configuracion,nombre_configuracion)
        self.fx=self.depth_intrinseca[0,0]
        self.calcular=np.poly1d(self.polinomio)
        self.maxi=2500
        # cargar modelo
        json_file = open("{}net.json".format(path_configuracion), 'r')
        loaded_model_json = json_file.read()
        json_file.close()
        loaded_model = model_from_json(loaded_model_json)
#         se cargan los pesos (weights) en el nuevo modelo
        loaded_model.load_weights("{}net.h5".format(path_configuracion))
        self.model=loaded_model
        print(self.model)
 
    def preprocesar_imgs(self,img_rgb,img_depth,f_segmentador,negro=True):
        #quitar distorsion z
        img_depth=self.distorsion_z(img_depth,filtrar=True)
     
        #quitar distorsion uv
#         img_rgb,img_depth=self.distorsion(img_rgb,img_depth)
        # cambiar de dominio rgb a depth
        img2_depth=self.rgb2_depth(img_rgb)
        # encontrar mascara
        ## hay que corregir 
        mascara,_=f_segmentador(img2_depth)
        return img_depth,mascara
    def calcular_volumen(self,img,mascara,img_ref):
#         graficar([img,img+(mascara*200),mascara],1,3,["imgz","img*mascara","mascara"])
        img_z=img.reshape(-1)
        mascara=mascara.reshape(-1)
        img_ref=img_ref.reshape(-1)
        # necesario para compatibilidad de dimensiones
        # extraer los pixeles del objetivo
        img_z=img_z[mascara==1]
        img_ref=img_ref[mascara==1]
        img_z=np.array(img_z,dtype=np.float32).reshape(-1,1)
        img_z2=np.power(img_z,2)
        img_z2=np.concatenate([img_z,img_z2],axis=1)
        #print("distancia maxima de profundidad en mm: {}".format(img_z.max()))
        tic()
        img_area=self.model.predict(self.normalizar(img_z2)).reshape(-1)
        print("time",toc())
#         print("area:",img_area.sum())
        img_z=img_z.reshape(-1)
        img_altura=img_ref-img_z
    
        
        img_volumen=img_area*img_altura
#         print("shape",img_volumen.shape,img_area.shape)
        volumen=np.sum(img_volumen)
        return volumen
    def calcular_error(self,valor_real,valor_medido):
#         print("eroror",valor_real,valor_medido)
        e=100*(valor_real-valor_medido)/valor_real
        return e
    def mejorar_referencia(self,img_ref):

#         img_ref[img_ref==0]=img_ref.mean()
        img_ref=self.distorsion_z(img_ref,1)

        return img_ref
    def abrir_img_ref(self,path,nombre):
        path_ref=os.path.join(path, '{}.png'.format(nombre))
        print(path_ref)
        ret=cv2.haveImageReader(path_ref)
        img_ref=0
        if ret:
            img_ref=cv2.imread(path_ref,cv2.IMREAD_UNCHANGED)[:,:,0]
            img_ref=self.mejorar_referencia(img_ref)
        return ret,img_ref
    def normalizar(self,IN,entrenar=False):
        
        IN[:,0]=IN[:,0]/self.maxi
        IN[:,1]=IN[:,1]/np.power(self.maxi,2)
        return IN
    

# calibración 3.a

In [ ]:
#
from ttictoc import tic,toc
from medicion import *
from tabulate import tabulate
import numpy as np
from keras.models import Sequential,model_from_json
from keras.layers import Dense, Activation
from keras.callbacks import ModelCheckpoint
import random
import matplotlib.pyplot as plt
class volumen3a(imagen):
    def __init__(self,path_configuracion,nombre_configuracion):
        imagen.__init__(self,path_configuracion,nombre_configuracion)

        # cargar modelo
        json_file = open("{}net.json".format(path_configuracion), 'r')
        loaded_model_json = json_file.read()
        json_file.close()
        loaded_model = model_from_json(loaded_model_json)
#         se cargan los pesos (weights) en el nuevo modelo
        loaded_model.load_weights("{}net.h5".format(path_configuracion))
        self.model=loaded_model
        
        self.fvo=self.depth_intrinseca[0,2]
        self.fuo=self.depth_intrinseca[1,2]
        self.shape=[480,640]
        self.fvo=self.depth_intrinseca[0,2]
        self.fuo=self.depth_intrinseca[1,2]
        self.U,self.V=self.calcularUV(self.shape)
        self.mini=300**2# Secuencia de números enteros sin duplicados
        self.maxi=2500**2
    def calcularUV(self,shape):
        v,u=np.mgrid[0:self.shape[0],0:self.shape[1]]
        u=np.array(u-self.fuo,dtype=np.float32)
        v=np.array(v-self.fvo,dtype=np.float32)
        u=u**2# distancias al cuadrado
        v=v**2
        u=((u-u.min())/(u.max()-u.min())).reshape(-1)
        v=((v-v.min())/(v.max()-v.min())).reshape(-1)
#         radio=np.sqrt(np.power(u,2)+np.power(v,2))
#         radio=(radio-radio.min())/(radio.max()-radio.min())
        return u,v
    
    def preprocesar_imgs(self,img_rgb,img_depth,f_segmentador,negro=True):
        #quitar distorsion z
        img_depth=self.distorsion_z(img_depth,filtrar=True)
     
        #quitar distorsion uv
#         img_rgb,img_depth=self.distorsion(img_rgb,img_depth)
        # cambiar de dominio rgb a depth
        img2_depth=self.rgb2_depth(img_rgb)
        # encontrar mascara
        ## hay que corregir 
        mascara,_=f_segmentador(img2_depth)
        return img_depth,mascara
    def calcular_volumen(self,img,mascara,img_ref):
#         graficar([img,img+(mascara*200),mascara],1,3,["imgz","img*mascara","mascara"])
        img_z=img.reshape(-1)
        mascara=mascara.reshape(-1)
        img_ref=img_ref.reshape(-1)
        # necesario para compatibilidad de dimensiones
        # extraer los pixeles del objetivo
        img_z=img_z[mascara==1]
        img_ref=img_ref[mascara==1]
        img_z2=self.normalizar(img_z.copy()).reshape(-1,1)
        u=self.U[mascara==1].reshape(-1,1)
        v=self.V[mascara==1].reshape(-1,1)
        IN=np.concatenate([img_z2,u,v],axis=1)
        #print("distancia maxima de profundidad en mm: {}".format(img_z.max()))
        tic()
        img_area=self.model.predict(IN).reshape(-1)
        print("time",toc())
#         print("area:",img_area.sum())
        img_z=img_z.reshape(-1)
        img_altura=img_ref-img_z
    
        
        img_volumen=img_area*img_altura
#         print("shape",img_volumen.shape,img_area.shape)
        volumen=np.sum(img_volumen)
        return volumen
    def calcular_error(self,valor_real,valor_medido):
        e=100*(valor_real-valor_medido)/valor_real
        return e
    def normalizar(self,zpx):
        
        zpx=zpx**2
        print(zpx.shape)
        zpx=(zpx-self.mini)/(self.maxi-self.mini)
        return zpx
    def desnormalizar(self,zpx):
        zpx=np.sqrt(zpx*(self.maxi-self.mini)+self.mini)
        return zpx
    def abrir_img_ref(self,path,nombre):
        path_ref=os.path.join(path, '{}.png'.format(nombre))
        print(path_ref)
        ret=cv2.haveImageReader(path_ref)
        img_ref=0
        if ret:
            img_ref=cv2.imread(path_ref,cv2.IMREAD_UNCHANGED)[:,:,0]
            img_ref=self.mejorar_referencia(img_ref)
        return ret,img_ref
    def mejorar_referencia(self,img_ref):

#         img_ref[img_ref==0]=img_ref.mean()
        img_ref=self.distorsion_z(img_ref,1)

        return img_ref

# calibración 4

In [ ]:
# calibración incluyendo la recuperación de la información 
from medicion import imagen
import sys# las sisguientes dos lineas ayudan a ubicar al archivo calibracion.py
sys.path.append("/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion")
from calibracion import *
import  os
class volumen4(imagen):
    def __init__(self,path_configuracion,nombre_configuracion):
      
        imagen.__init__(self,path_configuracion,nombre_configuracion,pendientes2=True)
        self.fx=self.depth_intrinseca[0,0]
        self.fy=self.depth_intrinseca[1,1]
        self.fvo=self.depth_intrinseca[0,2]
        self.fuo=self.depth_intrinseca[1,2]
        self.constante_area=1/(self.fx*self.fy)
        self.beta=np.array([ 5.83234140e-02,  9.53776679e-02,  3.02404506e-02, -5.51787924e+06,-1.65675186e+06])
        self.minZ=10000
        self.maxZ=25000
        self.U=[]
        self.V=[]
    def denormalizar(self, img):
        return img*(self.maxZ-self.minZ)+self.minZ
    def normalizarz(self, z):
        z=(z-self.minZ)/(self.maxZ-self.minZ)
        return z
    def normalizarUV(self,x):# recive el vector u o el vector v, solo uno a la ves
        x=(x-x.mean())/(x.std())
        return x
    def calcularUV(self,shape):
        v,u=np.mgrid[0:shape[0],0:shape[1]]
        self.U=np.array(u-self.fuo,dtype=np.float64)
        self.V=np.array(v-self.fvo,dtype=np.float64)
        self.U=self.normalizarUV(self.U)
        self.V=self.normalizarUV(self.V)
 
    def preprocesar_imgs(self,img_rgb,img_depth,f_segmentador):
        buffer=img_depth.copy()
        #recuperar información
        img_depth=self.recuperar(img_depth)
        #quitar distorsion z
        img_depth=self.distorsion_z2(img_depth,filtrar=True)
        graficar([img_depth,buffer],1,2)
        #quitar distorsion uv
        img_rgb,img_depth=self.distorsion(img_rgb,img_depth)
        # cambiar de dominio rgb a depth
        img2_depth=self.rgb2_depth(img_rgb)
        # encontrar mascara
        ## hay que corregir 
        mascara,img=f_segmentador(img2_depth)
        return img_depth,mascara
    def calcular_volumen(self,img,mascara,img_ref):
        if self.U==[]:
            shape=img.shape
            self.calcularUV(shape)
        print(mascara.shape)
        index=np.where(mascara==1)
        # acomodar datos para evaluar en la función
        z_mascara=self.normalizarz(img.astype(np.float64)[index].reshape(-1,1))# es necesario colocar la información dentro de un objeto lista
        u_mascara=self.U[index].reshape(-1,1)
        v_mascara=self.V[index].reshape(-1,1)
#         ref_masacara=self.normalizarz(img_ref.astype(np.float64)[index].reshape(-1,1))
        # estimar area de imagen
        img_area=self.beta[0]*z_mascara**2+(z_mascara*self.beta[1])*np.exp(-((u_mascara)/self.beta[3]+(v_mascara)/self.beta[4]))+self.beta[2]
        print(img_area.sum(),"area")
        img_altura=(img_ref.copy()-img.copy())/100
        img_altura=img_altura[index].reshape(-1,1)
        print("r",img_altura.mean())
        img_volumen=img_area*img_altura
        volumen=np.sum(img_volumen)
        print(volumen,"volumen")
        return volumen
    def calcular_error(self,valor_real,valor_medido):
        e=100*(valor_real-valor_medido)/valor_real
        return e
    def mejorar_referencia(self,img_ref):

        img_ref=self.recuperar(img_ref)
        img_ref=self.distorsion_z2(img_ref,1)
        return img_ref
    def abrir_img_ref(self,path,nombre):
        path_ref=os.path.join(path, '{}.png'.format(nombre))
      
        ret=cv2.haveImageReader(path_ref)
        img_ref=0
        if ret:
            img_ref=cv2.imread(path_ref,cv2.IMREAD_UNCHANGED)[:,:,0]
            img_ref=self.mejorar_referencia(img_ref)
        return ret,img_ref
    def recuperar(self,img):
        mask=img.copy()
        mask[mask<=300]=1
        mask[mask!=1]=0
        mask=mask.astype(np.uint8)
        dst = cv2.inpaint(img,mask,1,cv2.INPAINT_TELEA)
        dst=cv2.blur(dst,(3,3))
#         graficar([dst],1,1)
        return dst

In [ ]:
from tabulate import tabulate
from skimage import morphology
import cv2
import os
import sys# las sisguientes dos lineas ayudan a ubicar al archivo calibracion.py
sys.path.append("/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion")
from calibracion import *

def medir(path_config,list_config,path_img,valor_real,titulo="Error"):
    nxpaso=4
    list_titulos=["error porcentual"]
    list_tablas=[]
    ejex=[]
    ejey=[]
    for config in list_config:
        print(config)
        a=volumenb(path_config,config)
     
            # abrir el par de imagenes
        list_medidas=[]
        list_error=[]
        list_media=[]
        for i in range(1,70):
            print(i)
            ret,img_rgb,img_depth=a.abrir_par_img(path=path_img,
                                            indicador_rgb="gray",
                                            indicador_depth="depth",
                                            nombre=i
                                           )
            if ret:
                print(img_depth.mean())
#                 graficar([img_depth],1,1)
                if (i-1)%nxpaso==0:
                    ret,img_ref=a.abrir_img_ref(path_img,"ref_{}".format(i-1))
                    
                #segmentar
                img_depth,mascara=a.preprocesar_imgs(img_rgb,img_depth,segmetador)
                #calcular
                print("valor medio profundidad mm: ",img_depth.mean()/10)
                valor_calculado=a.calcular_volumen(img_depth/10,mascara,img_ref/10)
                
                error=a.calcular_error(valor_real,valor_calculado)
                print("error",error)
                list_medidas.append(valor_calculado)
                list_error.append(error)
                list_media.append(img_ref.max())
                print("valor medio profundidad mm: ",img_depth.mean()/10)
    #             print("valor real mm²: {}".format(valor_real))
    #             print("area calculada mm²:{}".format(area_calculada))
    #             print("porcentaje de error:{}".format(error))
    #     print(valor_real)
        tabla=zip(list_medidas,list_error,list_media)
        ejex.append(list_media)
        ejey.append(np.absolute(list_error))
        list_titulos.append(config)
#         print("valor real:{}".format(valor_real))
#         print(tabulate(tabla,headers=["v calculado","error","distancia media"]))
        print(" Error absoluto medio: ", np.absolute(np.array(list_error)).mean())
        plot([list_media],[np.absolute(list_error)],1,1,[titulo],[["Profundidad","Error absoluto %"]],tipo=1)
        return list_error,list_media
    
def segmetador(img,negro=True):
    n=200
    kerner=kernel = np.ones((n,n),np.uint8)
    img_gray=cv2.cvtColor(img.copy(), cv2.COLOR_RGB2GRAY)
    #img_= cv2.medianBlur(img_gray,15)
    if negro:
        criterio= cv2.THRESH_BINARY+cv2.THRESH_OTSU
        img_gray = cv2.morphologyEx(img_gray, cv2.MORPH_BLACKHAT, kernel)#MORPH_BLACKHAT
       
    else:
        criterio=cv2.THRESH_BINARY+cv2.THRESH_OTSU
    thr,mascara =cv2.threshold(img_gray,0,1,criterio)
    mascara.dtype=bool## remove_small solo funciona con variables binarias
    mascara=morphology.remove_small_objects(mascara,50,connectivity=10)
    mascara.dtype=np.uint8
    img_mascara=cv2.bitwise_and(img,img,mask=mascara)
    return mascara,img_mascara


# abrir img
img=cv2.imread("/home/estufab4/Desktop/flujo de bagazo/codigo/img_medicion/volumen/cubo2/gray_2.png")
mascara,img_mascara=segmetador(img,1)
# print("humbral: {}".format(thr))

graficar([img,mascara,img_mascara],1,3,["original","mascara","original X mascara"])

# medición de cubo

In [ ]:
# tabla0

In [ ]:
# medición de cubo
%matplotlib inline

path_img="/home/estufab4/Desktop/flujo de bagazo/codigo/img_medicion/volumen/cubo2/"
path_config="/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuraciónPrueba/"
# "/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuracion_union_150/"
list_config=["distorsionz"]
valor_real=1000100
cubo,list_media_cubo=medir(path_config,list_config,path_img,valor_real,titulo="cubo")
tabla0=np.array([[x,y] for x,y in zip(list_media_cubo,cubo) if x>=12000 and x<=18000])
tabla0=np.array(tabla0)

print("error absoluto medio: ",tabla0[:,1].mean())
print("error maximo:",tabla0[:,1].max())
print("error minimo:",tabla0[:,1].min())
plt.plot(tabla0[:,0],tabla0[:,1])
plt.show()

In [ ]:
# medición de cubo
%matplotlib inline

path_img="/home/estufab4/Desktop/flujo de bagazo/codigo/img_medicion/volumen/cubo2/"
path_config="/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuraciónPrueba/"
# "/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuracion_union_150/"
list_config=["distorsionz"]
valor_real=1000100
cubo,list_media_cubo=medir(path_config,list_config,path_img,valor_real,titulo="cubo")
tabla=np.array([[x,y] for x,y in zip(list_media_cubo,cubo) if x>=12000 and x<=18000])
tabla=np.array(tabla)

print("error absoluto medio: ",tabla[:,1].mean())
print("error maximo:",tabla[:,1].max())
print("error minimo:",tabla[:,1].min())
plt.plot(tabla[:,0],tabla[:,1])
plt.show()

# medición de bagazo

In [ ]:
from skimage import morphology
import numpy as np
import cv2
import sys# las sisguientes dos lineas ayudan a ubicar al archivo calibracion.py y segementador.py
sys.path.append("/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/")
sys.path.append("/home/estufab4/Desktop/flujo de bagazo/codigo/segmentacion/")
from calibracion import graficar, plot
from segmentacion import AbrirSegmentador
class calcular_volumenes(volumenb):
    def __init__(self,path_configuracion,nombre_configuracion):
        super().__init__(path_configuracion,nombre_configuracion)
 
    def abrir_imagenes(self,path,path_ref,nombre_img_ref):
        self.segmentador=AbrirSegmentador(pathConfig=path,
                                       nombre="config")
        list_nombres=[]
        list_img_depth=[]
        list_mascara=[]
        list_z_maximo=[]
        list_ref=[]
        _,img_ref=self.abrir_img_ref(path_ref,nombre_img_ref)
        #img_ref=imshow("{}ref_0.png".format(path),IMREAD_UNCHANGED)
        for i in range(0,70):
          
            ret,img_rgb,img_depth=self.abrir_par_img(path=path, indicador_rgb="gray",
                                            indicador_depth="depth",
                                            nombre=i
                                           )
            
            
            #verificar existencia
            if ret:
                #abrir img referencia 
               
                ret,img_ref=self.abrir_img_ref(path_ref,nombre_img_ref)
                print(img_ref.max())
               
                #quitar distorsión y segmentar
                img_depth,mascara=self.preprocesar_imgs(img_rgb,img_depth,self.segmentador.segmentar)
      
                list_nombres.append(i)
                list_img_depth.append(img_depth/10)
                list_mascara.append(mascara)
                list_z_maximo.append(img_depth.max())
                list_ref.append(img_ref/10)
            
        return list_nombres,list_img_depth,list_mascara,list_z_maximo,list_ref
    def medir(self,list_img_depth,list_mascara,list_ref,valor_real):
        list_medidas=[]
        list_error=[]
        for img_depth,mascara,img_ref in zip(list_img_depth,list_mascara,list_ref):
            volumen_calculado=self.calcular_volumen(img_depth,mascara,img_ref)
            error=self.calcular_error(valor_real,volumen_calculado)
            list_medidas.append(volumen_calculado)
            list_error.append(error)
        return list_medidas,list_error
    

In [ ]:
from tabulate import tabulate
import os


import matplotlib.pyplot as plt
# path_config="/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuracion_union_150/"
# list_configuracion=[
#              "configuracion_d5_90"
# #             "configuracion_d10_140",
# #             "configuracion_distorsion_d5_70",
# #             "configuracion_distorsion_d5_90"
# ]
# #   
path_config="/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuraciónPrueba/"
# "/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuracion_union_150/"
list_configuracion=["distorsionz"]
for config in list_configuracion:
        i=1
#     for i in range(1,10):
        volumenes=calcular_volumenes(path_config,config)
        path_img="/home/estufab4/Desktop/flujo de bagazo/codigo/img_medicion/bagazo/bagazo_1_5m/bagazosinfondo1_3/"
        path_img="/home/estufab4/Desktop/flujo de bagazo/codigo/img_medicion/bagazo/bagazo_1_5m_copy/bagazosinfondo1_1/"
        path_img="/home/estufab4/Desktop/flujo de bagazo/codigo/img_medicion/bagazo/bagazo_1_5m/recorte/"

        path_ref="/home/estufab4/Desktop/flujo de bagazo/codigo/img_medicion/bagazo/bagazo_1_5m/fondo/"
        path_ref="/home/estufab4/Desktop/flujo de bagazo/codigo/img_medicion/bagazo/bagazo_1_5m_copy/fondo/"
        nombre_img_ref="depth_{}".format(i)
        img=volumenes.abrir_img_ref(path_ref,nombre_img_ref)

        list_nombres,list_img_depth,list_mascara,list_z_maximo,list_ref=volumenes.abrir_imagenes(path_img,path_ref,nombre_img_ref)
        list_medidas,list_error=volumenes.medir(list_img_depth,list_mascara,list_ref,10000000)

        print("error max:",np.absolute(np.array(list_error)).max())
        print("error min:",np.absolute(np.array(list_error)).min())
        print("error promedio:",np.absolute(np.array(list_error)).mean())
        print("desviación estandar:",np.absolute(np.array(list_error)).std())
        plt.hist(np.array(list_error), bins='auto')
        plt.show()
        for i in range(len(list_nombres)):
                print(list_nombres[i])
                print("medición",list_medidas[i])
                print("error:",list_error[i])
                print(list_img_depth[i].max())
                img_z=cv2.bitwise_and(list_img_depth[i],list_img_depth[i],mask=list_mascara[i])
                graficar([list_img_depth[i],list_mascara[i],img_z],1,3)

In [ ]:
import numpy as np

a=np.array([[1,2],[2,3]])
a=a.reshape(-1,1)
b=np.concatenate([a,a],axis=1)
print(b)

In [ ]:
a